In [7]:
import pickle

#opening the batches.meta file for info on label names
with open('batches.meta', 'rb') as f:
    meta = pickle.load(f)

print(meta)

{'num_cases_per_batch': 10000, 'label_names': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], 'num_vis': 3072}


In [8]:
import pickle
import numpy as np
import os


# Combining all the data into one dataset
def load_cifar_batches(batch_folder, num_batches=5):

    images = []
    labels = []

    #looping through the folder and unpickling the datasets
    for i in range(1, num_batches + 1):
        batch_file = os.path.join(batch_folder, f"data_batch_{i}")
        with open(batch_file, 'rb') as f:
            batch = pickle.load(f, encoding = 'bytes')

        #extracting and processing data
        batch_images = batch[b'data'].reshape(10000, 3, 32, 32).astype('float32') / 255.0
        batch_labels = batch[b'labels']

        images.append(batch_images)
        labels.extend(batch_labels)

    #combining all batches into a single dataset & stacking images vertically
    images = np.vstack(images)
    return images, labels

#doing the same thing for testing data
def load_test_batch(batch_folder):

    batch_file = os.path.join(batch_folder, "test_batch")
    with open(batch_file, 'rb') as f:
        batch = pickle.load(f, encoding = 'bytes')

    images = batch[b'data'].reshape(10000, 3, 32, 32).astype('float32') / 255.0
    labels = batch[b'labels']
    return images, labels

#putting functions in action
batch_folder = "/Users/ryannyathi/Documents/CompIntelligence/Cifar_10_data"  # Path to the folder where the batches are stored
train_images, train_labels = load_cifar_batches(batch_folder)
test_images, test_labels = load_test_batch(batch_folder)

print(f"Train images: {train_images.shape}, Train labels: {len(train_labels)}")
print(f"Test images: {test_images.shape}, Test labels: {len(test_labels)}")


Train images: (50000, 3, 32, 32), Train labels: 50000
Test images: (10000, 3, 32, 32), Test labels: 10000


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

#converting to pytorch tensor

train_images_tensor = torch.from_numpy(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

#doing the same for testing data
test_images_tensor = torch.from_numpy(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)


#creating dataloaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#checking the output
print(f"Train loader batches: {len(train_loader)}, Test loader batches: {len(test_loader)}")


Train loader batches: 782, Test loader batches: 157


In [31]:
#creating cnn

class SimpleCNN(nn.Module):
    def __init__(self):

        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10) #final layer 128 neurons mapped to 10 output classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8) #flattening the array to create a vector for fc layers
        x = torch.relu(self.fc1(x))#passing flattened vector through fc1 with relu activation
        x = self.fc2(x)
        return x

model = SimpleCNN()

model(train_images_tensor).shape #batch size by number of classes




torch.Size([50000, 10])

In [14]:
#adding loss for classification and using adam optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr =0.001)


In [22]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")
    print("Training complete.")

    # Save the trained model
    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("Model saved.")

from matplotlib import pyplot as plt

def plot_training_loss(losses):
    plt.plot(losses)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.show()


train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch 1, Loss: 0.4631
Epoch 2, Loss: 0.4631
Epoch 3, Loss: 0.4632
Epoch 4, Loss: 0.4631
Epoch 5, Loss: 0.4633
Epoch 6, Loss: 0.4633
Epoch 7, Loss: 0.4635
Epoch 8, Loss: 0.4634
Epoch 9, Loss: 0.4633
Epoch 10, Loss: 0.4632
Training complete.
Model saved.


In [17]:
def evaluate_model(model, test_loader):
    model.eval()  #setting model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  #disabling gradient for evaluation
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  #getting predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model, test_loader)


Test Accuracy: 69.58%


In [18]:
torch.save(model.state_dict(), "baseline_model.pth")


In [1]:
#checking the weights of the gradient based trained model
model = SimpleCNN()
model.load_state_dict(torch.load("baseline_model.pth"))

#looping through the layers and print the weights
for name, param in model.named_parameters():
  print(f"Layer: {name}")
  print(f"Weights Shape: {param.shape}")
  print(f"Weights: {param}")
  print("-" * 20)

NameError: name 'SimpleCNN' is not defined